### Python Generator
Iterator is used for 
* read large data sets
* memory-intensive operations
* by a lazy item by item fashion

What an iterator is:
* maintain state. It doesn't know how many values to print, but knows what the next value is
* use lazy evaluation. Don't know the value until is triggered to do so
* doesn't store sequence in memory (space efficient)
* support Next() method which yields (grabs) the next value one at a time
* most iterables such as list or tuple has the iter() method that returns a generator

When you have a large dataset, it make sense to use lazy evaluation and only evalue one value at a time because store the dataset in memory is inefficient and often impossbile

A generator function returns a generator object
A generator object uses lazy evaluation to yield sequences
A generator refers to a generator object
A generator is an iterator, but not all iterators are generator objects

## Generator function
generator function returns generator objects using yield, and without using an list/containers

In [81]:
# a "normal" list/container function to generate a list
def even_integers_function(n):
    result = []
    for i in range(n):
        if i % 2 == 0:
            result.append(i)
    return result

even_integers_function(10)

[0, 2, 4, 6, 8]

In [82]:
# a generator function
def even_integers_generator(n):
    for i in range(n):
        if i % 2 == 0:
            yield i

In [83]:
even_integers_generator(10)

<generator object even_integers_generator at 0x7fbc44213ac0>

### How to consume a generator in python 3

In [84]:
# generate a generator object
even_generator = even_integers_generator(10)

#### Consume a generator by next() method

In [85]:
# next() method is no longer available for pyhton 3, use next(generator) instead
next(even_generator)

0

#### Consume a generator using for loop

In [86]:
for n in even_generator:
    print(n)

2
4
6
8


#### When a generator is consumed completely, it will stop iteration
if you use next(), it will generate an exception, but if you use a for loop, for loop has the mechanism to take care of it

In [87]:
# exception thrown
next(even_generator)

StopIteration: 

In [88]:
# no exception thrown
for n in even_generator:
    print(n)

In [89]:
# this is a list
[item.upper() for item in 'GeeksforGeeks']

# this is a generator using generator expression, which is more concise than generator function
(item.upper() for item in 'GeeksforGeeks')

# this is the generator expression for even numbers
(n for n in range(10) if n%2 == 0)


<generator object <genexpr> at 0x7fbc4427e7b0>

#### We can also directly consume the generator object generated by generator expression of function

In [90]:
for n in (i for i in range(10) if i %2 ==0):
    print(n)

0
2
4
6
8


In [91]:
for n in even_integers_generator(10):
    print(n)

0
2
4
6
8


### We can use generator expression to convert/process elements in a list
generator expression takes whatever the logic you can put between the parentheses to generate the generator

In [92]:
# Example 1
#list of mixed format numbers
numbers = [7, 22, 4.5, 99.7, '3', '5']

#convert numbers to integers using expression
integers = (int(n) for n in numbers)

In [93]:
integers

<generator object <genexpr> at 0x7fbc44220270>

In [94]:
# Example 2
# list of strings of names
names_list = ['Adam','Anne','Barry','Brianne','Charlie','Cassandra','David','Dana']

#Converts names to uppercase
uppercase_names = (name.upper() for name in names_list)

In [95]:
list(uppercase_names)

['ADAM', 'ANNE', 'BARRY', 'BRIANNE', 'CHARLIE', 'CASSANDRA', 'DAVID', 'DANA']

In [96]:
# generator expressions can be chained together
upper_case = (name.upper() for name in names_list)
reverse_uppercase = (name[::-1] for name in upper_case)
list(reverse_uppercase)

['MADA', 'ENNA', 'YRRAB', 'ENNAIRB', 'EILRAHC', 'ARDNASSAC', 'DIVAD', 'ANAD']

In [97]:
# a generator can be directly passed to a function without extra ()
max(i for i in range(10) if i %2 ==0)

8

### To summarize:
* generator objects cannot be reused
* calling next() on an exhausted generator raises StopIteration
* for loop handles StopIteration
* you can directly pass the generator to a function without the extra ()

### Creating a generator object for fibonacci sequence

In [98]:
# using the generator function
def fibonacci_gen():
    i = 1
    j = 1
    while True:
        yield j
        i, j = j, i+j
    

In [99]:
f = fibonacci_gen()

In [100]:
for i in range(10):
    print(next(f))

1
2
3
5
8
13
21
34
55
89


### build generator pipelines
* generators can be put together to build pipelines by linking several pipes
* items flow one by one through the entire pipeline
* pipeline functionality can be packaged into callable functions

In [101]:
def separate_names(names):
    for full_name in names:
        for name in full_name.split(' '):
            yield name

full_names = (name.strip() for name in open('names.txt'))
names = separate_names(full_names)
lengths = ((name, len(name)) for name in names)
longest = max(lengths, key=lambda x:x[1])
print(longest)

('Humbertson', 10)


This pipeline starts from the last step, and each time, one item is evaluated through the pipeline bottom up.

### Context Manager
Context manager is a python object that is able to act as a control structure when used after "with" statment. It manages the context for caller, including:
* set up for caller (try)
* yield control back to the caller (hand off) (yield)
* wrap-up when caller is done (finally)

Example: `With open('filename.txt'):
             #do something`

For a python object to be acted as a context manager, it must implement the methods "enter" and "exit"

### Generator for context manager
generator provides a short-hand way to produce a context manager. This is a try-yield-finally pattern.

`
@contextmanager
def simple_context_manager(n):
    try:
        # setup code
        yield # context manager is suspended, and control goes to the caller
    finally:
        # wrap up code`

In [108]:
# increments some_property by 1

from contextlib import contextmanager

@contextmanager
def simple_context_manager(obj):
    try:
        obj.some_property += 1
        yield
    finally:
        obj.some_property -= 1


For the above generator function, we have the following:
* this function has a yield statement, and therefore will return a generator object
* it has a contextmanager decorator, python enables it as a context manager by filling in __enter__ and __exit__ mehtods
* the context manager accepts an object. When called, the first thing is to increment some_property of that object by one
* after that, yield will pause the context manager, and pass the control back to the caller for it to do actions
* when caller is finished, the context manager will go ahead and wrap up by executing the commands in finally block. In this case, it will decrement some_property

Context manager makes easy execution of setup code and wrap up code with the help of python generators and context manager decorator

### An example of context manager using generator

In [109]:
@contextmanager
def simple_context_manager(obj):
    try:
        obj.some_property += 1
        yield
    finally:
        obj.some_property -= 1
        
        
class Simple_obj(object):
    def __init__(self, arg):
        self.some_property = arg

In [110]:
# step 1
obj = Simple_obj(5)
print(obj.some_property)

# step 2
with simple_context_manager(obj):
    print(obj.some_property)
    
# step 3
print(obj.some_property)

5
6
5


* step 1: create a Simple_obj instance with arg = 5, and print the some_property value, which is 5
* step 2: use context manager for Simple_obj. Once the context manager starts, it increment the some_property by 1 and returns the control back to the caller, which executes the print statement, and print 6
* step 3: after context manager closes, print the some_property again, it returns to value of 5

#### Summary:
* the @contextmanager is important, otherwise `AttributeError: __enter__` will be raised
* the contextmanager() function (here is obj.some_property +=1) is called after a with statement
* the indented "with" block (here is `print(obj.some_property)`)executes at yield statement

### function of yield in context manager using generator
* yield pass the control back to caller
* if we put a value/object to the right of yield, it will make that value/object available inside the with block of the caller

In [105]:
from time import time
from contextlib import contextmanager

HEADER = "this is the header \n"
FOOTER = "\nthis is the footer \n"


@contextmanager
def new_log_file(name):    
    try:
        logname = name
        f = open(logname, 'w')
        f.write(HEADER)
        yield f
    finally:
        f.write(FOOTER)
        print("logfile created")
        f.close()

In [106]:
with new_log_file('logfile') as file:
    file.write("this is the body")

logfile created


In [107]:
!cat logfile

this is the header 
this is the body
this is the footer 


#### execution of the above code example:
* new_log_file context manager creates the file using the input string as file name, and write HEADER
* yield statement pass the control back to caller, together with the created file inside "with" statement
* caller inside "with" statement write "this is body" to the file
* context manager write the FOOTER and close the file

### Coroutines
* A generator produces values one at each time. A coroutine consumes values. It may or may not return values. Coroutine is not for iterate sequences
* A coroutine is built from a generator, but it is conceptually different. A coroutine is designed to 
    + repeatedly recieves input
    + processes input
    + stops at yield statement
    + send() is the method added to generators exclusively for coroutines
    + in coroutine, the yield statement is used to capture the value passed to send() method, and pauses flow
    + A coroutine must support send() method, and maintain internal state as a powerful tool in data process
* A coroutine is also different from a function
    + the same function is called each time 
    + coroutine has persistent properties that can be changed and altered. It can change the state of its own properties, the state of something else, or both

In [111]:
def coroutine_example():
    while True:
        x = yield
        #do something
        print(x)

In [112]:
# create a coroutine object
c = coroutine_example()

In [113]:
# if we directly consume, we get error messsage.
c.send(10)

TypeError: can't send non-None value to a just-started generator

In [114]:
# we need to prime the coroutine by calling next()
# here the value send to coroutine is what the yield statement becomes
# since x is set to yield, we can use x 
next(c)

In [115]:
c.send(10)

10


In [117]:
def counter(string):
    count = 0
    try:
        while True:
            item = yield
            if isinstance(item, str):
                if item in string:
                    count += 1
                    print(item)
                else:
                    print('No Match')
            else:
                print('Not a string')
    except GeneratorExit:
        print(count)

In [121]:
# create a coroutine
c = counter('California')

# prime it
next(c)

# send value, which will be sent by yield to item, and check the match
c.send('Cali')
c.send('nia')
c.send('Hawaii')
c.send(1234)
c.close()

2
Cali
nia
No Match
Not a string
2


#### Using a coroutine decorator to do the prime automatically
* the decorator accepts a generator function, and returns a wrap function that does prime using next(cr)

In [123]:
def coroutine_decorator(func):
    def wrap(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr)
        return cr
    return wrap


@coroutine_decorator
def coroutine_example():
    while True:
        x = yield
        #do something with x
        print(x)

In [124]:
c = coroutine_example()
c.send(10)

10


#### Different coroutines can be used to handle different data processes
* the same sender function can apply different coroutines to the same/different files
* separate the logic of open and close files from filtering and counting it

In [125]:
def sender(filename, target):
    for line in open(filename):
        target.send(line)
    target.close()
        

@coroutine_decorator
def match_counter(string):
    count = 0
    try:
        while True:
            line = yield
            if string in line:
                count += 1
    except GeneratorExit:
        print('{}: {}'.format(string, count))



@coroutine_decorator
def longer_than(n):
    count = 0
    try:
        while True:
            line = yield
            if len(line)>n:
                print(line)
                count += 1
    except GeneratorExit:
        print('longer than {}: {}'.format(n, count))


In [127]:
m = match_counter('Da')
sender("names.txt", m)

Da: 5


In [128]:
l = longer_than(14)
sender("names.txt", l)

Armanda Pilling

Lahoma Mondragon

Karina Vandyne

Calista Overbay

Cherise Retana

Lashanda Demoura

Bong Humbertson

Vasiliki Stonge

Danita Vallance

Salena Dulmage

Sergio Cockrill

Delaine Creamer

Clarita Jetter

Tanesha Finkbeiner

Leonore Cushman

Dwain Mccotter

Alphonse Bellomy

Deadra Bisceglia

Josefine Montijo

Brittanie Talamantes

Carolina Powel

Octavio Trumbull

longer than 14: 22


### Coroutine pipeline
* The whole process is driven from the begining, and not pulled from the end process as in generator pipeline
* Data is pushed to the pipeline
* a coroutine can send data in different directions (both receive and send data)
* doesn't rely on multiple end targets to pull data through

In [130]:
@coroutine_decorator
def router():
    try:
        while True:
            line = yield
            (first, last) = line.split(' ')
            fnames.send(first)
            lnames.send(last.strip())
    except GeneratorExit:
        fnames.close()
        lnames.close()
        
        

@coroutine_decorator
def file_write(filename):
    try:
        with open(filename,'a') as file:
            while True:
                line = yield
                file.write(line+'\n')
    except GeneratorExit:
        file.close()
        print('one file created')

if __name__ == "__main__":
    fnames = file_write('first_names.txt')
    lnames = file_write('last_names.txt')
    router = router()
    for name in open('names.txt'):
        router.send(name)
    router.close()
        

one file created
one file created


#### The process is the following:
* fnames and lnames are two coroutines that create the files, and write the received data (using send()) to the file
* router is a coroutine that split the received string names and send to fnames and lnames